Model of a generalized Josephson Junction Array with Majorana Bound States (MBS)

Authors: Cliff Sun

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import os

In [2]:
def generate_random_numbers(n):
    random_numbers = [random.uniform(0, 1) for _ in range(n)]
    return random_numbers

# Example: Generate 5 random numbers
num_of_junctions = 30
random_numbers = np.array(np.sort(generate_random_numbers(2 * num_of_junctions)))

In [3]:
# TODO: Implement 2d array algo for calculating critical current of MBS
# TODO: Implement free variable for M1 in MBS

In [4]:
arrayOfJunctions = [0, 0.001, 0.999, 1]

In [5]:
def even_arrayOfWidths_noInput(sigma, numOfJunctions, width):
    delta = 4 / (numOfJunctions + 2) * sigma**2
    arrayOfWidths = np.zeros(numOfJunctions)
    arrOfWidthsDiv2 = np.zeros(numOfJunctions//2)
    # Creates the list of the terms generated by the delta, without accounting for the mean
    # NOT THE TRUE STANDARD DEVIATION - Need to account for both positives and minuses, this is just positive
    for i in range(1, numOfJunctions//2 + 1):
        arrOfWidthsDiv2[i - 1] = np.sqrt(i * delta)
    arrayOfWidths = np.concatenate((np.flip(-1 * arrOfWidthsDiv2), arrOfWidthsDiv2))
    minimum = min(arrayOfWidths)
    for i in range (len(arrayOfWidths)):
        arrayOfWidths[i] -= (minimum - width)
    return arrayOfWidths

In [6]:
def EvenArrayOfJunctions(sigma, numOfJunctions, width, arrayJ = []): # Generates an array of junctions given some standard deviation & mean width
    # Necessary Declarations
    junctionCenter = 1/(numOfJunctions - 1) # Declares the middle of the junction assuming zero junction width
    arrOfJunctions =  np.zeros(numOfJunctions * 2)
    arrayOfWidths = []
    if (sigma == 0 and len(arrayJ) != 0):
        return arrayJ
    arrayOfWidths = even_arrayOfWidths_noInput(sigma, numOfJunctions, width)
    # If the user inputted a custom array
    rannum = int(random.randint(0, numOfJunctions - 1))
    arr = np.zeros(numOfJunctions)
    arrOfJunctions[1] = arrayOfWidths[rannum]
    arr[rannum] = 1
    for k in range(2, 2*numOfJunctions - 2,2):
        while(arr[rannum] == 1):
            rannum = int(random.randint(0, numOfJunctions - 1))
        arrOfJunctions[k] = k//2 * junctionCenter - (arrayOfWidths[rannum]/2)
        arrOfJunctions[k + 1] = k//2 * junctionCenter + (arrayOfWidths[rannum]/2)
        arr[rannum] = 1

    for i in range(numOfJunctions):
        if (arr[i] == 0):
            arrOfJunctions[-2] = 1 - arrayOfWidths[i]
            arrOfJunctions[-1] = 1
            break
    
    return arrOfJunctions

def criticalCurrent(density, arrJ):
    criticalCurrents = []
    junctionWidths = []
    for i in range(len(arrJ)//2):
        junctionWidths.append(arrJ[2*i+1] - arrJ[2*i])
    for i in range (len(junctionWidths)):
        criticalCurrents.append(junctionWidths[i] * density[i])
    return criticalCurrents, junctionWidths

In [7]:
criticalCurrents, junctionWidths = criticalCurrent(np.ones(len(arrayOfJunctions)//2), arrayOfJunctions)

prints out the elements in the junction in a better format

In [8]:
index = 0
while (index < (len(arrayOfJunctions) - 1)):
    if (index == len(arrayOfJunctions) - 2):
        print(str(arrayOfJunctions[index]) + " - " + str(arrayOfJunctions[index + 1]), end = " ")
    else:
        print(str(arrayOfJunctions[index]) + " - " + str(arrayOfJunctions[index + 1]) + ",", end = " ")
    index += 2

0 - 0.001, 0.999 - 1 

Parameters for Current -> Magnetic Field, Junction Locations, Critical Currents, Initial Phase Difference

In [9]:
def JJ_MBScurrent(B, arrJ, arrC, phi_0, MBS_Amplitude, ABSs): # y is initial phase difference of the whole circuit, B is the magnetic field, arrJ is the location of junctions, arrC is critical current associated with each junction
    curr = 0 # summation of all currents in the entire junction
    limit = int(len(arrJ) / 2) # number of junctions in the SQUID
    numOfSegments = 0
    for n in range(limit):
        numOfSegments = 5 if (arrJ[2 * n + 1] - arrJ[2 * n]) < 0.05 else int(100 * (arrJ[2 * n + 1] - arrJ[2 * n]))
        sizeOfSegment = float((arrJ[2 * n + 1] - arrJ[2 * n]) / numOfSegments)
        for i in range(numOfSegments):
            curr += arrC[n] * (np.sin(phi_0 + (2 * np.pi * B) * (arrJ[2 * n] + i * sizeOfSegment) + ABSs[n])) * (1/numOfSegments) + arrC[n] * MBS_Amplitude * np.sin(phi_0 + ((2 * np.pi * B) * (arrJ[2 * n] + i * sizeOfSegment))/2) * (1/numOfSegments)
    return curr

In [10]:
# def MBScurrent(B, arrJ, arrC, MBS_phase, MBS_Amplitude): # y is initial phase difference of the whole circuit, B is the magnetic field, arrJ is the location of junctions, arrC is critical current associated with each junction
#     curr = 0 # summation of all currents in the entire junction
#     limit = int(len(arrJ) / 2) # number of junctions in the SQUID
#     numOfSegments = 0
#     for n in range(limit):
#         numOfSegments = 5 if (arrJ[2 * n + 1] - arrJ[2 * n]) < 0.05 else int(100 * (arrJ[2 * n + 1] - arrJ[2 * n]))
#         sizeOfSegment = float((arrJ[2 * n + 1] - arrJ[2 * n]) / numOfSegments)
#         for i in range(numOfSegments):
#             curr += arrC[n] * MBS_Amplitude * np.sin(MBS_phase + ((2 * np.pi * B) * (arrJ[2 * n] + i * sizeOfSegment))/2  )* (1/numOfSegments)
#     return curr

In [11]:
def maxJJ_MBSCurrent(B, arrayJ, arrayC, MBS_Amplitude, ABSs): # Spits out the maximum current by varying the gauge invariant phase of the left end (free parameter) gamma
    Y=np.linspace(0, 4*np.pi, 300)
    dummyArray=[]
    for gamma in Y:
        dummyArray.append(JJ_MBScurrent(B, arrayJ, arrayC, phi_0=gamma, MBS_Amplitude=MBS_Amplitude, ABSs=ABSs))
    return max(dummyArray)

In [12]:
# def maxMBSCurrent(B, arrayJ, arrayC, MBS_Amplitude): # Spits out the maximum current by varying the gauge invariant phase of the left end (free parameter) gamma
#     Y=np.linspace(0, 4*np.pi, 300)
#     dummyArray=[]
#     for gamma in Y:
#         dummyArray.append(MBScurrent(B, arrayJ, arrC=arrayC, MBS_phase=gamma, MBS_Amplitude=MBS_Amplitude))
#     return max(dummyArray)

def array_to_python_syntax(arr):
    if not arr:
        return "[]"

    result = "["
    for item in arr:
        if isinstance(item, str):
            result += f"'{item}', "
        else:
            result += f"{item}, "
    
    # Remove the trailing comma and space
    result = result[:-2]
    result += "]"
    return result

def write_txt_file(directory, arrJ, MBS, file_name):
    numOfJunctions = len(arrJ)//2
    converted_arrJ = array_to_python_syntax(arrJ)
    text_content = f"Majorana Bound State Amplitude = {MBS}. \n \n The Junction array is {converted_arrJ}."
    text_file_path = os.path.join(directory, file_name)
    with open(text_file_path, 'w') as file:
        file.write(text_content)

def create_folder(repository_path, folder_name):
    folder_path = os.path.join(repository_path, folder_name)
    # Check if the folder already exists
    if not os.path.exists(folder_path):
        # Create the folder
        os.makedirs(folder_path)
        print(f"Folder '{folder_name}' created successfully.")
    else:
        print(f"Folder '{folder_name}' already exists.")
    return folder_path

def plot_MBS_graphs(arr_MBS, arr_ABS, arrJ, directory):
    criticalCurrents, junctionWidths = criticalCurrent(np.ones(len(arrJ)//2), arrJ=arrJ)
    MagField = np.linspace(-10, 10, 10000)
    numOfJunctions = len(arrJ)//2
    for MBS in arr_MBS:
        general_name = f"{numOfJunctions}_{MBS}_{np.std(junctionWidths)}"
        folder_path = create_folder(repository_path=directory, folder_name=general_name)
        norm = np.sum(criticalCurrents) * (MBS + 1)
        Ic = []
        for i in range(len(MagField)):
            Ic.append(maxJJ_MBSCurrent(MagField[i], arrJ, criticalCurrents, MBS_Amplitude=MBS, ABSs=arr_ABS) / norm)
        plt.plot(MagField, Ic)
        plt.xlabel('Flux')
        plt.ylabel('Ic')
        plt.xlim(-10, 10)
        plt.ylim(0, 1)
        plt.savefig(f"{folder_path}/{general_name}.png")
        write_txt_file(directory=folder_path, arrJ=arrJ, MBS=MBS, file_name=f"{general_name}.txt")
        plt.clf()


In [13]:
# MagField = np.linspace(-10, 10, 10000) # an array of Magnetic Fields ranging from 0 to 100 with 5000 total elements

In [14]:
IMaxPoint = []
MBS_Amplitude = 0.1
arrayOfJunctions = [0, 0.001, 0.999, 1]
ABSs = [0, 0]

In [15]:
# length = len(MagField)
# for i in range(length):
#     IMaxPoint.append(maxJJ_MBSCurrent(MagField[i], arrayOfJunctions, criticalCurrents, MBS_Amplitude=MBS_Amplitude, ABSs=ABSs) / (np.sum(criticalCurrents) * (MBS_Amplitude + 1))) # This integer represents the number of segments you want to cut each junction up into (the higher the number, the better the approximation)

In [16]:
# plt.plot(MagField, IMaxPoint, 'g-')
# plt.xlabel('B')
# plt.ylabel('Critical Current')  
# # plt.ylim(0,3) 
# plt.xlim(-10,10)
# plt.grid()            
# plt.show()
# # plt.savefig('10-random-junctions:B=-5to5.png')

In [17]:
# print(max(IMaxPoint))

In [18]:
# print(min(IMaxPoint))

In [19]:
# print(np.mean(junctionWidths))

In [20]:
# print(np.std(junctionWidths))

In [21]:
# diff = []

# for n in range(int(len(IMaxPoint)/2)):
#     diff.append(np.abs(IMaxPoint[int(len(MagField)/2)-n] - IMaxPoint[int(len(MagField)/2)+n]))

In [22]:
# plt.plot(MagField[int(len(MagField)/2):], diff)
# plt.xlabel('|B|')
# plt.ylabel('|I(-B) - I(B)|')
# # plt.ylim(0,1)
# plt.grid()
# plt.show()

In [23]:
directory = r"MBS_Results"
MBS_array = np.linspace(0, 1, 11)
plot_MBS_graphs(arr_MBS=MBS_array, arr_ABS=ABSs, arrJ=arrayOfJunctions, directory=directory)

Folder '2_0.0_4.336808689942018e-19' created successfully.


Folder '2_0.1_4.336808689942018e-19' created successfully.
Folder '2_0.2_4.336808689942018e-19' created successfully.
Folder '2_0.30000000000000004_4.336808689942018e-19' created successfully.
Folder '2_0.4_4.336808689942018e-19' created successfully.
Folder '2_0.5_4.336808689942018e-19' created successfully.
Folder '2_0.6000000000000001_4.336808689942018e-19' created successfully.
Folder '2_0.7000000000000001_4.336808689942018e-19' created successfully.
Folder '2_0.8_4.336808689942018e-19' created successfully.
Folder '2_0.9_4.336808689942018e-19' created successfully.
Folder '2_1.0_4.336808689942018e-19' created successfully.


<Figure size 640x480 with 0 Axes>